In [1]:
import os, getpass

In [2]:
os.environ["OPENAI_API_KEY"] = getpass.getpass("Ingresa tu API Key de OpenAI : ")

In [3]:
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import OpenAIWhisperParser
from langchain.document_loaders.blob_loaders.youtube_audio import YoutubeAudioLoader

url = "https://www.youtube.com/watch?v=5yyB6qmafuc" #5 minutos
save_dir = "docs/youtube/"
loader = GenericLoader(
    YoutubeAudioLoader([url], save_dir),
    OpenAIWhisperParser()
)
data = loader.load()

[youtube] Extracting URL: https://www.youtube.com/watch?v=5yyB6qmafuc
[youtube] 5yyB6qmafuc: Downloading webpage
[youtube] 5yyB6qmafuc: Downloading ios player API JSON
[youtube] 5yyB6qmafuc: Downloading android player API JSON
[youtube] 5yyB6qmafuc: Downloading m3u8 information
[info] 5yyB6qmafuc: Downloading 1 format(s): 140
[download] Destination: docs/youtube//Los límites físicos de la inteligencia artificial.m4a
[download] 100% of    4.64MiB in 00:00:00 at 5.07MiB/s   
[FixupM4a] Correcting container of "docs/youtube//Los límites físicos de la inteligencia artificial.m4a"
[ExtractAudio] Not converting audio docs/youtube//Los límites físicos de la inteligencia artificial.m4a; file is already in target format m4a
Transcribing part 1!


In [4]:
data

[Document(page_content='¿Qué pasaría si la inteligencia artificial, la que está detrás de nuestro feed de redes sociales, de los pagos bancarios o de las búsquedas de Google, se detiene? ¿Qué pasa si deja de funcionar? Estamos todavía lejos de que eso pase, pero lo que sí está claro es que, antes de que la inteligencia artificial se apodere del planeta, es más fácil quedarnos sin recursos físicos para entrenar y hacer funcionar esta tecnología, que va creciendo más rápido de lo que todos esperaban. Aquí les cuento más. La computación, los cálculos computacionales de los programas de inteligencia artificial, normalmente se hacen en centros de datos. Esos son edificios que albergan miles de computadoras y servidores. Pero esta capacidad física, obvio, también puede llegar a sus límites, de seguir usando la inteligencia artificial al ritmo al que lo venimos haciendo. Esto nos deja con dos opciones, o construir más centros de datos, o hacer que la computación sea más eficiente. Y cualquier

In [10]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    chunk_size = 400, 
    chunk_overlap = 20
)

docs = text_splitter.split_documents(documents = data)

## Cargando datos a Qrant

In [11]:
url = "https://ebe8df73-f101-4c3f-8edd-385af9c77530.us-east4-0.gcp.cloud.qdrant.io:6333" #Reemplaza por tu url
api_key = getpass.getpass("Ingresa tu API Key de Qdrant : ")

In [12]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Qdrant

embedding = OpenAIEmbeddings()

vectorstore = Qdrant.from_documents(
    docs,
    embedding,
    url = url,
    api_key = api_key,
    collection_name = "c_youtube",
    force_recreate = True
)

## Recuperando contexto de la colección en Qdrant

In [13]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

template = """Responda a la pregunta basada en el siguiente contexto.
Si no puedes responder a la pregunta, usa la siguiente respuesta "No lo sé disculpa, puedes buscar la información en internet."

Contexto: 
{context}
Pregunta: {question}
Respuesta: 
"""

PROMPT = PromptTemplate(
    template = template, input_variables = ["context", "question"]
)

llm = ChatOpenAI(
    model_name = 'gpt-3.5-turbo',
    temperature = 0.0
)

qa = RetrievalQA.from_chain_type(
    llm = llm,
    chain_type = "stuff",
    retriever = vectorstore.as_retriever(),
    verbose = True,
    chain_type_kwargs = {"prompt": PROMPT, "verbose": True}
)

In [14]:
qa.run("cuáles son los límites físicos de la IA")



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Responda a la pregunta basada en el siguiente contexto.
Si no puedes responder a la pregunta, usa la siguiente respuesta "No lo sé disculpa, puedes buscar la información en internet."

Contexto: 
¿Qué pasaría si la inteligencia artificial, la que está detrás de nuestro feed de redes sociales, de los pagos bancarios o de las búsquedas de Google, se detiene? ¿Qué pasa si deja de funcionar? Estamos todavía lejos de que eso pase, pero lo que sí está claro es que, antes de que la inteligencia artificial se apodere del planeta, es más fácil quedarnos sin recursos físicos para entrenar y hacer funcionar esta tecnología, que va creciendo más rápido de lo que todos esperaban. Aquí les cuento más. La computación, los cálculos computacionales de los programas de inteligencia artificial, normalmente se hacen en centros de datos. Esos son edificios que albe

'Los límites físicos de la IA incluyen la capacidad limitada de los centros de datos para realizar cálculos computacionales, la limitación en la disponibilidad de recursos físicos como electricidad y agua, y la limitación en la capacidad de reducir el tamaño de los transistores y enfriarlos en los microchips.'

In [15]:
qa.run("de qué trata la ley de Moore")



> Entering new RetrievalQA chain...


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Responda a la pregunta basada en el siguiente contexto.
Si no puedes responder a la pregunta, usa la siguiente respuesta "No lo sé disculpa, puedes buscar la información en internet."

Contexto: 
¿Qué pasaría si la inteligencia artificial, la que está detrás de nuestro feed de redes sociales, de los pagos bancarios o de las búsquedas de Google, se detiene? ¿Qué pasa si deja de funcionar? Estamos todavía lejos de que eso pase, pero lo que sí está claro es que, antes de que la inteligencia artificial se apodere del planeta, es más fácil quedarnos sin recursos físicos para entrenar y hacer funcionar esta tecnología, que va creciendo más rápido de lo que todos esperaban. Aquí les cuento más. La computación, los cálculos computacionales de los programas de inteligencia artificial, normalmente se hacen en centros de datos. Esos son edificios que albe


> Finished chain.

> Finished chain.

> Finished chain.


'La ley de Moore se refiere a la observación de que el número de transistores en un chip de computadora se duplica aproximadamente cada dos años.'